# Hyper-Parameter Tuning with Keras, PipelineAI, and MLflow


In [ ]:
import tensorflow as tf

from keras.models import Model
from keras.callbacks import Callback
from keras.layers import Dense, Flatten
import mlflow
from datetime import datetime
import time
import math
import numpy as np
import os
import yaml
import click
import matplotlib.pyplot as plt
from keras.utils import plot_model

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## MLflow Logger

In [ ]:
class MLflowLogger(Callback):
    """
    Keras callback for logging metrics and final model with MLflow.

    Metrics are logged after every epoch. The logger keeps track of the best model based on the
    validation metric. At the end of the training, the best model is logged with MLflow.
    """
    def __init__(self, model, x_train, y_train, x_test, y_test,
                 **kwargs):
        self._model = model
        self._best_val_loss = math.inf
        self._train = (x_train, y_train)
        self._valid = (x_test, y_test)
        self._pyfunc_params = kwargs
        self._best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        """
        Log Keras metrics with MLflow. Update the best model if the model improved on the validation
        data.
        """
        if not logs:
            return
        for name, value in logs.items():
            if name.startswith("val_"):
                name = "valid_" + name[4:]
            else:
                name = "train_" + name
            mlflow.log_metric(name, value)
        val_loss = logs["val_loss"]
        if val_loss < self._best_val_loss:
            # Save the "best" weights
            self._best_val_loss = val_loss
            self._best_weights = [x.copy() for x in self._model.get_weights()]

    def on_train_batch_begin(self, *args, **kwargs):
        pass
            
    def on_train_batch_end(self, *args, **kwargs):
        pass
    
    def on_train_end(self, *args, **kwargs):
        """
        Log the best model with MLflow and evaluate it on the train and validation data so that the
        metrics stored with MLflow reflect the logged model.
        """
        self._model.set_weights(self._best_weights)
        x, y = self._train
        train_res = self._model.evaluate(x=x, y=y)
        for name, value in zip(self._model.metrics_names, train_res):
            mlflow.log_metric("train_{}".format(name), value)
        x, y = self._valid
        valid_res = self._model.evaluate(x=x, y=y)
        for name, value in zip(self._model.metrics_names, valid_res):
            mlflow.log_metric("valid_{}".format(name), value)


## Specify `user_id`
* `<YOUR_USER_ID>`  - 8 character id that uniquely identifies the PipelineAI user.  You will see the UserId in the upper right hand corner of the Settings tab after you login to [PipelineAI Community Edition](https://community.cloud.pipeline.ai)

![user-id](https://pipeline.ai/assets/img/user-id.png)

In [ ]:
#user_id = <YOUR_USER_ID>
user_id = ''

model_name = 'mnist'
model_tag = 'v1'

tracking_uri = 'https://community.cloud.pipeline.ai'
    
def run(epochs_list, batch_size_list):
    mlflow.set_tracking_uri(tracking_uri)

    experiment_name = '%s%s-%s' % (user_id, model_name, model_tag)
    
    # This will create and set the experiment
    mlflow.set_experiment(experiment_name)

    for epochs in epochs_list:
        for batch_size in batch_size_list:
            with mlflow.start_run() as run:
                print('***')
                print('***')
                print('*** Experiment:  %s' % (experiment_name))
                print('*** Run Id:  %s' % (run.info.run_uuid))
                print('*** Training with epochs %s and batch_size %s' % (epochs, batch_size))
                print('***')
                
                mlflow.log_param('epochs', str(epochs))
                mlflow.log_param('batch_size', str(batch_size))

                mnist = tf.keras.datasets.mnist

                (x_train, y_train), (x_test, y_test) = mnist.load_data()
                x_train, x_test = x_train / 255.0, x_test / 255.0

                model = tf.keras.models.Sequential([
                  tf.keras.layers.Flatten(input_shape=(28, 28)),
                  tf.keras.layers.Dense(512, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(0.2),
                  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                ])

                model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

                history = model.fit(
                    x=x_train,
                    y=y_train,
                    validation_data=(x_test, y_test),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[MLflowLogger(model=model,
                                            x_train=x_train,
                                            y_train=y_train,
                                            x_test=x_test,
                                            y_test=y_test,
                                            domain={},
                                            artifact_path="model",
                                            image_dims=(28, 28))
                              ])

                model.evaluate(x_test, y_test)

                saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./mnist_model/pipeline_tfserving")
                if type(saved_model_path) != str:
                    saved_model_path = saved_model_path.decode('utf-8')

                # From the following:  https://keras.io/visualization/
                print(history)
                print(history.history)
 
                # Plot training & validation accuracy values
                plt.plot(history.history['acc'])
                plt.plot(history.history['val_acc'])
                plt.title('Model Accuracy')
                plt.ylabel('Accuracy')
                plt.xlabel('Epoch')
                plt.legend(['Train', 'Test'], loc='upper left')
                plt.show()
                plt.savefig('mnist_model/viz_training_accuracy.png')

                # Plot training & validation loss values
                plt.plot(history.history['loss'])
                plt.plot(history.history['val_loss'])
                plt.title('Model Loss')
                plt.ylabel('Loss')
                plt.xlabel('Epoch')
                plt.legend(['Train', 'Test'], loc='upper left')
                plt.show()
                plt.savefig('./mnist_model/viz_training_loss.png')

                plot_model(model, to_file='./mnist_model/viz_pipeline_model.png')

                mlflow.log_artifacts(saved_model_path)
                mlflow.log_artifact('./mnist_model/pipeline_conda_environment.yaml')
                mlflow.log_artifact('./mnist_model/pipeline_train.py') 
                mlflow.log_artifact('./mnist_model/pipeline_invoke_python.py')
                mlflow.log_artifact('./mnist_model/pipeline_modelserver.properties')
                mlflow.log_artifact('./mnist_model/pipeline_tfserving.properties')
                mlflow.log_artifact('./mnist_model/MLproject')
                mlflow.log_artifact('./mnist_model/pipeline_condarc')
                mlflow.log_artifact('./mnist_model/pipeline_ignore')
                mlflow.log_artifact('./mnist_model/pipeline_setup.sh')
                mlflow.log_artifact('./mnist_model/viz_pipeline_model.png')
                mlflow.log_artifact('./mnist_model/viz_training_accuracy.png')
                mlflow.log_artifact('./mnist_model/viz_training_loss.png')            

                mlflow.end_run()

    print('***')
    print('***')
    print('*** Completed Experiment: %s' % (experiment_name))
    print('***')

## Call `run()` with any combination of `epochs_list` and `batch_size_list`

In [ ]:
run(epochs_list=[2, 5], batch_size_list=[64, 128])